<a href="https://colab.research.google.com/github/salmanarif86/MLAI/blob/master/event3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Cup Event #3

In [0]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
import sklearn
import spacy
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.22.2.post1.


In [6]:
import os
os.getcwd()

'/content'

In [9]:
df = pd.read_csv("spamraw_train.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5000 non-null   int64 
 1   sms_text  5000 non-null   object
 2   spam      5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


,id,sms_text,spam
0,1,Hope you are having a good week. Just checking in,0
1,2,K..give back my thanks.,0
2,3,Am also doing in cbe only. But have to pay.,0
3,4,"complimentary 4 STAR Ibiza Holiday or £10,000 ...",1
4,5,okmail: Dear Dave this is your final notice to...,1


In [10]:
df.count()

id          5000
sms_text    5000
spam        5000
dtype: int64

In [12]:
nlp = spacy.load('en_core_web_lg')

OSError: ignored

In [0]:
from sklearn.model_selection import train_test_split

X = df['sms_text']
y = df['spam']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vectorizer = CountVectorizer(max_features = 20)
vectorizer.fit(X_train)

dtm_train = vectorizer.transform(X_train)
dtm_train.shape

dtm_val = vectorizer.transform(X_val)
dtm_val.shape

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=20, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

(4000, 20)

(1000, 20)

In [0]:
feature_names = vectorizer.get_feature_names()
feature_names

['and',
 'are',
 'call',
 'can',
 'for',
 'have',
 'in',
 'is',
 'it',
 'me',
 'my',
 'now',
 'of',
 'on',
 'so',
 'that',
 'the',
 'to',
 'you',
 'your']

# Build Model

In [0]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=2)
clf.fit(dtm_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

# Estimate Model Performance

In [0]:
from sklearn.metrics import confusion_matrix

pred_val = clf.predict(dtm_val)

confusion_matrix(y_val, pred_val)

array([[843,  27],
       [ 75,  55]], dtype=int64)

In [0]:
from sklearn.metrics import classification_report, f1_score

print("F1 Score = {:.2f}".format(f1_score(y_val, pred_val)))
print()
print(classification_report(y_val, pred_val))

F1 Score = 0.52

              precision    recall  f1-score   support

           0       0.92      0.97      0.94       870
           1       0.67      0.42      0.52       130

    accuracy                           0.90      1000
   macro avg       0.79      0.70      0.73      1000
weighted avg       0.89      0.90      0.89      1000



# Create Submission File on "Real" Test Data

In [0]:
test_df = pd.read_csv('sms-spam/spamraw_test.csv')

# Extract/create the features the same way you did for training
test_dtm = vectorizer.transform(test_df['sms_text'])

# Use your model to make predictions
pred_test = clf.predict(test_dtm)

# We will look at the predicted prices to ensure we have something sensible.
print(pred_test)

my_submission = pd.DataFrame({'id': test_df.id, 'predicted': pred_test})

my_submission.head()

# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1
 0 0 0 0 0 0 0 1 0 0 0 1 

,id,predicted
0,12000,1
1,12001,0
2,12002,0
3,12003,0
4,12004,0
